In [4]:
import asyncio
import nest_asyncio
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool as langchain_tool
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor


load_dotenv()

llm = ChatOpenAI(
    model="qwen3-max-2026-01-23",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0,
)


## --- 定义工具 ---
@langchain_tool
def search_information(query: str) -> str:
    """
    提供有关给定主题的事实信息。使用此工具查找诸如“法国首都”或“伦敦的天气？”等短语的答案。
    """
    print(f"\n--- 🛠️ 工具调用：search_information，查询：'{query}' ---")

    # 使用预定义结果字典模拟搜索工具
    simulated_results = {
        "weather in london": "伦敦目前多云，温度为 15℃。",
        "capital of france": "法国的首都是巴黎。",
        "population of earth": "地球的估计人口约为 80 亿人。",
        "tallest mountain": "珠穆朗玛峰是海拔最高的山峰。",
        "default": f"'{query}' 的模拟搜索结果：未找到特定信息，但该主题似乎很有趣。",
    }

    result = simulated_results.get(query.lower(), simulated_results["default"])
    print(f"--- 工具结果：{result} ---")
    return result


tools = [search_information]

## --- 创建工具调用 Agent ---
# 此提示词模板需要一个 `agent_scratchpad` 占位符用于 Agent 的内部步骤。
agent_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个有用的助手。"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# 创建 Agent，将 LLM、工具和提示词绑定在一起
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# AgentExecutor 是调用 Agent 并执行所选工具的运行时。
# 这里不需要 'tools' 参数，因为它们已经绑定到 Agent。
agent_executor = AgentExecutor(agent=agent, verbose=True, tools=tools)


async def run_agent_with_tool(query: str):
    """使用查询调用 Agent 执行器并打印最终响应。"""
    print(f"\n--- 🏃 使用查询运行 Agent：'{query}' ---")
    try:
        response = await agent_executor.ainvoke({"input": query})
        print("\n--- ✅ 最终 Agent 响应 ---")
        print(response["output"])
    except Exception as e:
        print(f"\n🛑 Agent 执行期间发生错误：{e}")


async def main():
    """并发运行所有 Agent 查询。"""
    tasks = [
        run_agent_with_tool("法国的首都是什么？"),
        run_agent_with_tool("伦敦的天气怎么样？"),
        run_agent_with_tool("告诉我一些关于狗的事情。"),  # 应该触发默认工具响应
    ]
    await asyncio.gather(*tasks)


nest_asyncio.apply()
asyncio.run(main())


--- 🏃 使用查询运行 Agent：'法国的首都是什么？' ---

--- 🏃 使用查询运行 Agent：'伦敦的天气怎么样？' ---

--- 🏃 使用查询运行 Agent：'告诉我一些关于狗的事情。' ---


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `search_information` with `{'query': '法国首都'}`



--- 🛠️ 工具调用：search_information，查询：'法国首都' ---
--- 工具结果：'法国首都' 的模拟搜索结果：未找到特定信息，但该主题似乎很有趣。 ---
'法国首都' 的模拟搜索结果：未找到特定信息，但该主题似乎很有趣。
Invoking: `search_information` with `{'query': '伦敦的天气'}`



--- 🛠️ 工具调用：search_information，查询：'伦敦的天气' ---
--- 工具结果：'伦敦的天气' 的模拟搜索结果：未找到特定信息，但该主题似乎很有趣。 ---
'伦敦的天气' 的模拟搜索结果：未找到特定信息，但该主题似乎很有趣。
Invoking: `search_information` with `{'query': '狗'}`



--- 🛠️ 工具调用：search_information，查询：'狗' ---
--- 工具结果：'狗' 的模拟搜索结果：未找到特定信息，但该主题似乎很有趣。 ---
'狗' 的模拟搜索结果：未找到特定信息，但该主题似乎很有趣。法国的首都是巴黎。

> Finished chain.

--- ✅ 最终 Agent 响应 ---
法国的首都是巴黎。
很抱歉，我无法提供伦敦当前的天气信息。建议您查看可靠的天气预报网站或应用程序（如BBC Weather、Weather.com等）以获取最新和准确的天气情况！

> Finished chain.

--- ✅ 最终 Agent 响应 ---
很抱歉，我无法提供伦敦当前的天气信息。建议您查看可靠的天气预报网站或应用程序